## HW Assignment                                                                                    
                                                                                           Michele Giovanni Calvi

In [1]:
import pandas as pd
import numpy as np

In [2]:
# getting all values of proteins for LabelEncoder
protein = {"TTT" : "F", "CTT" : "L", "ATT" : "I", "GTT" : "V",
           "TTC" : "F", "CTC" : "L", "ATC" : "I", "GTC" : "V",
           "TTA" : "L", "CTA" : "L", "ATA" : "I", "GTA" : "V",
           "TTG" : "L", "CTG" : "L", "ATG" : "M", "GTG" : "V",
           "TCT" : "S", "CCT" : "P", "ACT" : "T", "GCT" : "A",
           "TCC" : "S", "CCC" : "P", "ACC" : "T", "GCC" : "A",
           "TCA" : "S", "CCA" : "P", "ACA" : "T", "GCA" : "A",
           "TCG" : "S", "CCG" : "P", "ACG" : "T", "GCG" : "A",
           "TAT" : "Y", "CAT" : "H", "AAT" : "N", "GAT" : "D",
           "TAC" : "Y", "CAC" : "H", "AAC" : "N", "GAC" : "D",
           "TAA" : "STOP", "CAA" : "Q", "AAA" : "K", "GAA" : "E",
           "TAG" : "STOP", "CAG" : "Q", "AAG" : "K", "GAG" : "E",
           "TGT" : "C", "CGT" : "R", "AGT" : "S", "GGT" : "G",
           "TGC" : "C", "CGC" : "R", "AGC" : "S", "GGC" : "G",
           "TGA" : "STOP", "CGA" : "R", "AGA" : "R", "GGA" : "G",
           "TGG" : "W", "CGG" : "R", "AGG" : "R", "GGG" : "G" 
           }
values = set(protein.values())


In [3]:
# this function converts the DNA into proteins. As the hint suggests, it improves the NN performance. 
def translate(dna, protein):  # from https://www.geeksforgeeks.org/dna-protein-python-3/

    protein_sequence = ""

    # Generate protein sequence
    for i in range(0, len(dna)-(3+len(dna)%3), 3):
        if protein[dna[i:i+3]] == "STOP" :
            break
        protein_sequence += protein[dna[i:i+3]]
    return protein_sequence

# Uploading Data

In [4]:
data = np.load("dataset.npy").item()
xs = data["genes"] # [n_sample, arbitrary length string object]
ys = data["resistant"] # [n_sample, bool]

In [5]:
# Building dataframe
data = {"DNA": xs, "Label":ys}
df = pd.DataFrame(data)

In [6]:
df["DNA"] = df["DNA"].apply(lambda x: translate(x,protein)) 

In [7]:
df.head(5)

,DNA,Label
0,MHYRMIHWMMEIDCNGCANNTLSRRWNYDFWHKHVEQVKCYRHNIR...,False
1,MHYRMIHWMMEIDCNGCANNTLSRRWNYDFWHKHVEQVKCYRHNIR...,False
2,MHYRMIHWMMEIDCNGCANNTLSRRWNYDFWHKHVEQVKCYRHNIR...,False
3,MHYRMIHWMMEIDCNGCANNTLSRRWNYDFWHKHVEQVKCYRHNIR...,False
4,MHYRMIHWMMEIDCNGCANNTLSRRWNYDFWHKHVEQVKCYRHNIR...,True


In [8]:
df.describe(), df.info() #visualizing type and if exist missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
DNA      100000 non-null object
Label    100000 non-null bool
dtypes: bool(1), object(1)
memory usage: 879.0+ KB


(                                                      DNA   Label
 count                                              100000  100000
 unique                                              23699       2
 top     MHYRMIHWMMEIDCNGCANNTLSRRWNYDFWHKHVEQVKCYRHNIR...   False
 freq                                                 8524   50089, None)

In [9]:
#checking that the data is not imbalanced
len(df[df["Label"]==True])

49911

In [10]:
minimum = df['DNA'].apply(len).min()
maximum = df['DNA'].apply(len).max()
minimum, maximum

(2, 301)

Since I will solve the problem using an recurrent neural network it is necessary to decide a proper value for the sequence length. The problem with vanilla RNN is that with long sequences they tend to lose information due to the vanish gradient problem. This occurs when the gradient becomes so small that it will have little to no impact on the training of the weights of the neural networks. A solution to this problem would be using LSTMs. They solve this problem by controlling what should be stored and what should be deleted. These controls are called gates. They allow the LSTM to store information for longer sequence problems. 

For this problem all the sequence of 301  may be used and for the smaller proteins a padding preprocessing may be applied to obtain the same dimensions. However, being computationally expensive for my machine I have decided to use a sequence of 250 and lose some accuracy. 


In [11]:
df["DNA"] = df["DNA"].apply(lambda x: x[0:250])

# Categorical Features

To explain the encoding process  I will be using the DNA nucleobases even though I will be working with proteins. Note, that the explanation holds for proteins too.

The DNA is composed by the letters A, C, G and T. Since they cannot be used to train the model it is necessary to encode them. I could encode them such that A = 0, C = 1, G = 2 and T = 3. However, the model might give more importance to some of the nucleobases over the others. Hence, a one hot encoding will be applied. Of course the same discussion holds for the proteins, instead of having A,C,G,T we have the variables in the dictionary of _def (translate)_

This will create a 4 feature vector for every nucleobase, ex: 

A = [1 0 0 0]

C = [0 1 0 0]

G = [0 0 1 0]

T = [0 0 0 1]


However, the last column can be removed. The algorithm will understand that an input of [0 0 0] will be equivalent to T. However, since we will have to do some padding eventually which adds [0 0 0] it is necessary to keep the last column so that the LSTM understands the difference betweent the padded value and T. 

Hence, padded value = [0 0 0 0] and T = [0 0 0 1]. 



In [12]:
# Preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [13]:
df["DNA"] = df["DNA"].apply(lambda x: np.array(list(x)))


In [14]:
# Applying Label encoder
encoder = LabelEncoder()
encoder.fit(list(values))
one_hot_fit = encoder.transform(list(values)) #will later fit the one hot encoder so I am sure of taking all values
df["DNA"] = df["DNA"].apply(lambda x: encoder.transform(x))
df.iloc[1,0].shape

(250,)

In [15]:
# One Hot encoder 
df["DNA"] = df["DNA"].apply(lambda x: x.reshape(len(x),1))
onehot = OneHotEncoder(sparse=False, dtype=int, n_values=len(values))
onehot.fit(one_hot_fit.reshape(-1,1))
df["DNA"] = df["DNA"].apply(lambda x: onehot.transform(x))

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


In [16]:
# verifying shape
df.iloc[1,0].shape

(250, 21)

# LSTM Model

In [17]:
# Visualizing the data
df["Label"] = df["Label"].replace([True,False],[1,0])
df.head()

,DNA,Label
0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",0
1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",0
2,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",0
3,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",0
4,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,...",1


In [18]:
from sklearn.model_selection import train_test_split

# extracting the data from the dataframe and splitting into training and testing data sets
x = df.iloc[:,0].values
y = df.iloc[:,-1].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [19]:
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.
/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [20]:
# we have array of arrays but LSTM need tensor [# of samples, #timesteps, #features]

x_tr = []
for i in range(len(x_train)):
    x_tr.append(x_train[i])
x_t = []
for i in range(len(x_test)):
    x_t.append(x_test[i])
    
# Padding
x_tr = pad_sequences(x_tr) #makes all time steps the same
x_t = pad_sequences(x_t)   
x_tr.shape                 

(80000, 250, 21)

In this case we have that in both training and test set we have a data with the maximum time steps allowed (250). Hence the padding can be done seperately without risking of having two different sequences for train and test.

In [21]:
x_tr = x_tr[0:60000,:]
y_train = y_train[0:60000]#used 60,000 because slow training on my computer


In [22]:
import collections
collections.Counter(y_train) # verifying that also downsampling is still balanced

Counter({1: 29860, 0: 30140})

In [23]:
# parameters and libraries
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation

batch = 100
epoch = 2 #after several trainings I have noticed that the NN converges after roughly 2 epochs

### First Model

In [24]:
# First Model - used to understand after how many epochs the NN converges 
model = Sequential()
model.add(LSTM(16,input_shape = x_tr.shape[1:],batch_size = batch))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(optimizer = "Adam", loss = "binary_crossentropy",metrics=['accuracy'])

In [25]:
model.fit(x_tr,y_train, batch_size = batch, epochs=epoch,verbose=1)

Epoch 1/2
60000/60000 [==============================] - 90s 1ms/step - loss: 0.2681 - acc: 0.9302
Epoch 2/2
60000/60000 [==============================] - 88s 1ms/step - loss: 0.1926 - acc: 0.9421


In [26]:
scores = model.evaluate(x_t, y_test, batch_size=batch)
scores

20000/20000 [==============================] - 11s 539us/step


[0.18325849369168282, 0.943600001335144]

# Tuning Parameters
In this section I will be tuning the parameters using a K-fold cross validation

In [150]:

def build_model(optimizer,neurons):

    model = Sequential()
    model.add(LSTM(neurons,input_shape = x_tr.shape[1:]))
    model.add(Dense(1))
    model.add(Activation("sigmoid"))
    model.compile(optimizer = optimizer , loss = "binary_crossentropy",metrics=['accuracy'])
    
    return model

In [151]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [152]:
# making model compatible with scikit-learn
classifier = KerasClassifier(build_fn = build_model, batch_size = 100, epochs = epoch)

I am tuning the number of neurons in the LSTM layer and the type of optimizer. Other tunings can be made, for example the number of epochs, batches, learning rates. Due to time constraints I have decided to opt for the number of neurons and optimizer only

In [153]:
neurons = np.random.randint(2,128,3)
parameters = [{"optimizer": ["RMSprop"],"neurons": neurons},
             {"optimizer": ["Adam"],"neurons": neurons}]

In [154]:
# Grid Search with the parameters chosen
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters, scoring = 'accuracy', cv = 5) #5-fold CV
# Fit on training set
grid_search = grid_search.fit(x_tr, y_train)

Epoch 1/2
48000/48000 [==============================] - 112s 2ms/step - loss: 0.2891 - acc: 0.9262
Epoch 2/2
48000/48000 [==============================] - 108s 2ms/step - loss: 0.2000 - acc: 0.9412
Epoch 1/2
48000/48000 [==============================] - 107s 2ms/step - loss: 0.3281 - acc: 0.9101
Epoch 2/2
48000/48000 [==============================] - 106s 2ms/step - loss: 0.2118 - acc: 0.9393
Epoch 1/2
48000/48000 [==============================] - 110s 2ms/step - loss: 0.2917 - acc: 0.9112
Epoch 2/2
48000/48000 [==============================] - 107s 2ms/step - loss: 0.2051 - acc: 0.9391
Epoch 1/2
48000/48000 [==============================] - 118s 2ms/step - loss: 0.3269 - acc: 0.9027
Epoch 2/2
48000/48000 [==============================] - 121s 3ms/step - loss: 0.2070 - acc: 0.9400
Epoch 1/2
48000/48000 [==============================] - 119s 2ms/step - loss: 0.3112 - acc: 0.9107
Epoch 2/2
48000/48000 [==============================] - 106s 2ms/step - loss: 0.2049 - acc: 0.9415


In [155]:
#Viewing best parameters in Grid Search
best_parameter = grid_search.best_params_
best_accuracy = grid_search.best_score_ #best cros validated mean
print('Best parameter: ' + str(best_parameter))
print('Best accuracy: ' + str(best_accuracy))

Best parameter: {'neurons': 71, 'optimizer': 'Adam'}
Best accuracy: 0.9422666666666667


## Final Model 
Using parameters from 5-fold CV

In [156]:
final = Sequential()
final.add(LSTM(best_parameter["neurons"],input_shape = x_tr.shape[1:],batch_size = batch))
final.add(Dense(1))
final.add(Activation("sigmoid"))
final.compile(optimizer = best_parameter["optimizer"], loss = "binary_crossentropy",metrics=['accuracy'])

In [159]:
# choice of 16 neurons. 1,2 too small. 16 converges with less epochs
# explainw y sigmoid, loss = binary_cross_entropy
best_parameter["neurons"]
final.fit(x_tr,y_train, batch_size = batch, epochs=epoch,verbose=1)

Epoch 1/2
60000/60000 [==============================] - 119s 2ms/step - loss: 0.2420 - acc: 0.9294
Epoch 2/2
60000/60000 [==============================] - 102s 2ms/step - loss: 0.1867 - acc: 0.9425


In [160]:
scores = final.evaluate(x_t, y_test, batch_size=batch)

20000/20000 [==============================] - 13s 645us/step


[0.1775519283115864, 0.943600001335144]

In [161]:
scores[1]

0.943600001335144

# Criteria

I have tried several combinations for the architecture of the NN and saw small improvements when increasing the complexity of the architecture. Therefore, I have decided to stick to a simple architecture with one LSTM input layer and one output layer. 

The very first training model has 16 input neurons because when I using smaller values (1,2,4,8) the training's accuracy was lower. After several trials I found 16 to be the best value. Furthermore, I have opted for a binary crossentropy loss function because this is a binary classification problem. The advantage of this loss function is that it greatly penalizes bad predictions by using a negative log of the probability.  A sigmoid function is used as the output activation function of the NN as it provides the probability of the output belonging to a "TRUE" label. A softmax funciton could have been used, but it would require to use two outputs. And this being a binary classification we just need one output probability as the other can be obtained by: 

P(FALSE) = 1 - P(TRUE). 

The biggest challenges I have faced in the problem were not related to the architecture but more on how to preprocess the data. Choosing the right number of time steps and encoding all the variables. By using a simple Label encoder the performances of the NN were somewhat simialr to one hot encoding, with the difference that it took longer to train. Hence, before starting the k-fold CV I applied a one hot encoder as it would perform the cross validation in less time. 

# Comparing Performances

Training score - 94.25%

5-fold CV score - 94.23%

Test score - 94.36%

From the three diferent scores it is possible to see that the model doesn't underfit or overfit. The test score is the same as training score, a little bit higher. What causes this might be the fact that there are 23699 (from **Uploading Data**) combinations of proteins. Since we have 100000 samples it is possible to say that every data point is repeated roughly 4 times.